In [1]:
import pandas as pd

In [5]:
# set fred api key
fred_api_key = "653ed88f4bdd6c12141a8c80288845c4"
cpi_series_id = "CPIAUCSL"

In [12]:
# get cpi data
cpi = pd.read_json(f"https://api.stlouisfed.org/fred/series/observations?series_id={cpi_series_id}&api_key={fred_api_key}&file_type=json")

# make a dataframe from the observations
cpi = pd.DataFrame(cpi["observations"].values.tolist())

# select the date and value columns
cpi = cpi[["date", "value"]]
cpi.columns = ["date", "cpi"]

# convert date to datetime and cpi to numeric
cpi["date"] = pd.to_datetime(cpi["date"])
cpi['cpi'] = pd.to_numeric(cpi['cpi'])

# generate a yoy change column
cpi["cpi_yoy"] = cpi["cpi"].pct_change(12)

cpi

,date,cpi,cpi_yoy
0,1947-01-01,21.480,NaN
1,1947-02-01,21.620,NaN
2,1947-03-01,22.000,NaN
3,1947-04-01,22.000,NaN
4,1947-05-01,21.950,NaN
...,...,...,...
929,2024-06-01,313.049,0.029756
930,2024-07-01,313.534,0.029236
931,2024-08-01,314.121,0.025912
932,2024-09-01,314.686,0.024075


In [16]:
# get zori data
zori = pd.read_csv("zori_metro_long_clean.csv")

# select only us data
zori = zori[zori["name"] == "United States"]

# restrict to cols of interest
zori = zori[["date", "zori"]]

# convert date to datetime
zori["date"] = pd.to_datetime(zori["date"])

zori

,date,zori
0,2015-01-31,1219.505157
648,2015-02-28,1226.110978
1296,2015-03-31,1235.157406
1944,2015-04-30,1244.364461
2592,2015-05-31,1253.736804
...,...,...
73224,2024-06-30,1999.099587
73872,2024-07-31,2007.481254
74520,2024-08-31,2011.654608
75168,2024-09-30,2012.727325


In [17]:
zori.dtypes

date    datetime64[ns]
zori           float64
dtype: object

In [18]:
# date is currently the last day of the month, so we need to shift it to the first day of the month
zori["date"] = zori["date"] + pd.offsets.MonthBegin(-1)
zori

,date,zori
0,2015-01-01,1219.505157
648,2015-02-01,1226.110978
1296,2015-03-01,1235.157406
1944,2015-04-01,1244.364461
2592,2015-05-01,1253.736804
...,...,...
73224,2024-06-01,1999.099587
73872,2024-07-01,2007.481254
74520,2024-08-01,2011.654608
75168,2024-09-01,2012.727325


In [19]:
# add yoy change column
zori["zori_yoy"] = zori["zori"].pct_change(12)
zori

,date,zori,zori_yoy
0,2015-01-01,1219.505157,NaN
648,2015-02-01,1226.110978,NaN
1296,2015-03-01,1235.157406,NaN
1944,2015-04-01,1244.364461,NaN
2592,2015-05-01,1253.736804,NaN
...,...,...,...
73224,2024-06-01,1999.099587,0.035110
73872,2024-07-01,2007.481254,0.034336
74520,2024-08-01,2011.654608,0.033546
75168,2024-09-01,2012.727325,0.033218


In [21]:
# merge the two dataframes
# we want to keep all the dates in the zori dataframe
# so we use a left join
rent_inflation = pd.merge(
    zori,
    cpi,
    on="date",
    how="left"
    )

rent_inflation = rent_inflation[["date", "zori_yoy", "cpi_yoy"]]

rent_inflation

,date,zori_yoy,cpi_yoy
0,2015-01-01,NaN,-0.002299
1,2015-02-01,NaN,-0.000870
2,2015-03-01,NaN,-0.000220
3,2015-04-01,NaN,-0.001040
4,2015-05-01,NaN,0.000350
...,...,...,...
113,2024-06-01,0.035110,0.029756
114,2024-07-01,0.034336,0.029236
115,2024-08-01,0.033546,0.025912
116,2024-09-01,0.033218,0.024075


In [24]:
# melt the dataframe
rent_inflation = rent_inflation.melt(id_vars="date", var_name="metric", value_name="yoy_change")

In [25]:
# save to csv
rent_inflation.to_csv("rent_inflation.csv", index=False)